# 1. Setup

In [1]:
import pandas as pd
import re
import os

# 2. load

In [2]:
def read_data(file_path):
    try:
        df = pd.read_csv(file_path, delimiter="|", engine="python", 
                         on_bad_lines="skip", dtype=str)  # Đọc dưới dạng chuỗi để tránh lỗi kiểu dữ liệu
        print("Đọc dữ liệu thành công.")
        return df
    except Exception as e:
        print(f"Đọc dữ liệu thất bại: {e}")
        return None

file_path = r"D:\HTP\Project-personal\PriceCar\dataset\raw\car.csv"
df = read_data(file_path)

if df is not None:
    df.head(2)


Đọc dữ liệu thành công.


# 3. Transform

In [3]:
# Loại bỏ các cột không cần thiết
def remove_unnecessary_columns(df, columns_to_remove):
    df.drop(columns=columns_to_remove, inplace=True, errors='ignore')
    print(f"Đã loại bỏ các cột: {columns_to_remove}")
    return df

columns_to_remove = ['km_1', 'imp_exp_1']
if df is not None:
    df = remove_unnecessary_columns(df, columns_to_remove)



# Đổi tên cột 'price' thành 'price_words'
def rename_column(df, old_name, new_name):
    df.rename(columns={old_name: new_name}, inplace=True)
    print(f"Đã đổi tên cột '{old_name}' thành '{new_name}'")
    return df

if df is not None:
    df = rename_column(df, 'price', 'price_words')



# Loại bỏ dấu gạch ngang và dấu gạch dưới trong các cột kiểu chuỗi
def clean_string_columns(df):
    for col in df.select_dtypes(include=['object']).columns:
        if col != 'poster_tel':  # Giữ nguyên dấu gạch ngang trong 'poster_tel'
            df[col] = df[col].str.replace('-', ' ').str.replace('_', ' ')
    print("Đã loại bỏ dấu gạch ngang và dấu gạch dưới ")
    return df

if df is not None:
    df = clean_string_columns(df)



# Chuyển đổi cột 'price_words' thành 'price_num' với đơn vị VNĐ
def convert_price(price):
    match = re.match(r'(\d+)\s*(Triệu|Tỷ)', price)
    if match:
        value, unit = match.groups()
        value = int(value.replace('.', '')) 
        if unit == 'Tỷ':
            value *= 1000000000  
        elif unit == 'Triệu':
            value *= 1000000  
        return value
    return None

def create_price_num_column(df, price_column='price_words', new_column='price_num'):
    df[new_column] = df[price_column].apply(convert_price)
    print(f"Đã tạo cột '{new_column}' từ cột '{price_column}'")
    return df

# Tạo cột 'price_num'
if df is not None:
    df = create_price_num_column(df)


# Loại bỏ các hàng có giá trị null
def remove_null_rows(df):
    df.dropna(inplace=True)
    print("Đã loại bỏ các hàng có giá trị null.")
    return df

if df is not None:
    df = remove_null_rows(df)

# Loại bỏ các hàng có giá trị rỗng trong bất kỳ cột chuỗi nào
def remove_empty_rows(df):
    df = df[~df.apply(lambda row: row.astype(str).str.strip().eq('').any(), axis=1)]
    print("Đã loại bỏ các hàng có giá trị rỗng.")
    return df

if df is not None:
    df = remove_empty_rows(df)



# Sắp xếp lại thứ tự các cột 
def reorder_columns(df, column_order):
    df = df[column_order]
    print("Đã sắp xếp lại các cột.")
    return df

# Định nghĩa thứ tự cột mới
column_order = ['car_model', 'km', 'imp_exp', 'car_type', 'out_color', 'in_color', 'door_num', 'seat_num', 
                'new_old', 'car_year', 'title', 'price_words', 'price_num', 'area', 'poster_name', 
                'poster_add', 'poster_tel']

if df is not None:
    df = reorder_columns(df, column_order)



# Loại bỏ các khoảng trắng thừa trong các cột 
def remove_extra_spaces(df):
    for col in df.select_dtypes(include=['object']).columns:
        df[col] = df[col].str.replace(r'\s+', ' ', regex=True).str.strip()
    print("Đã loại bỏ các khoảng trắng thừa trong các cột ")
    return df

if df is not None:
    df = remove_extra_spaces(df)


Đã loại bỏ các cột: ['km_1', 'imp_exp_1']
Đã đổi tên cột 'price' thành 'price_words'
Đã loại bỏ dấu gạch ngang và dấu gạch dưới 
Đã tạo cột 'price_num' từ cột 'price_words'
Đã loại bỏ các hàng có giá trị null.
Đã loại bỏ các hàng có giá trị rỗng.
Đã sắp xếp lại các cột.
Đã loại bỏ các khoảng trắng thừa trong các cột 


# 4. normalize data

In [4]:
def normalize_data(df):
    columns_to_normalize_int = ['km', 'door_num', 'seat_num', 'car_year', 'price_num']
    
    for col in columns_to_normalize_int:
        df[col] = df[col].astype(str)  # Đảm bảo cột là kiểu chuỗi
        df[col] = df[col].str.replace(r'\D', '', regex=True)  # Loại bỏ tất cả ký tự không phải là chữ số
        df[col] = df[col].replace('', '0')  # Thay thế các giá trị trống bằng '0'
        df[col] = df[col].astype(int)  


    for col in df.select_dtypes(include=['object']).columns:
        if col not in columns_to_normalize_int:  
            df[col] = df[col].astype(str)
        
    print("Dữ liệu đã được chuẩn hóa.")
    return df

if df is not None:
    df = normalize_data(df)


Dữ liệu đã được chuẩn hóa.


# 5. Check

In [5]:
# Kiểm tra kiểu dữ liệu của các cột
def check_data_types(df):
    print(df.dtypes)

if df is not None:
    check_data_types(df)


car_model      object
km              int64
imp_exp        object
car_type       object
out_color      object
in_color       object
door_num        int64
seat_num        int64
new_old        object
car_year        int64
title          object
price_words    object
price_num       int64
area           object
poster_name    object
poster_add     object
poster_tel     object
dtype: object


In [6]:
# Kiểm tra số lượng giá trị null trong mỗi cột
def check_nulls(df):
    null_counts = df.isnull().sum()
    if null_counts.sum() == 0:
        print("Không còn giá trị null trong DataFrame.")
    else:
        print(f"Có giá trị null trong các cột sau:\n{null_counts[null_counts > 0]}")
    return null_counts

if df is not None:
    check_nulls(df)


Không còn giá trị null trong DataFrame.


# 6. Save

In [7]:
# Lưu dữ liệu đã transform vào file CSV mới
def save_data(df, output_file):
    try:
        if os.path.exists(output_file):
            os.remove(output_file)  # Kiểm tra nếu file đã tồn tại thì xóa
        df.to_csv(output_file, index=False, encoding='utf-8')
        print(f"Dữ liệu đã được lưu tại: {output_file}")
    except Exception as e:
        print(f"Không thể lưu dữ liệu: {e}")

output_file = r"D:\HTP\Project-personal\PriceCar\dataset\processed\car.csv"

if df is not None:
    save_data(df, output_file)


Dữ liệu đã được lưu tại: D:\HTP\Project-personal\PriceCar\dataset\processed\car.csv
